То, что нам понадобится

In [5]:
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import cross_val_score
from operator import itemgetter as ig
from functools import partial

Функция, вытаскивающая цену из строки, фильтрует все символы, кроме цифр

In [ ]:
def parse_price(val):
    return float(''.join(filter(str.isdigit,val)))/1000000


df_train = pd.read_hdf("../input/train_data.h5")
df_train['price'] = df_train['price'].map(parse_price)
df_train['logprice'] = np.log(df_train['price'])
df_test = pd.read_hdf("../input/test_data.h5")

Объединяем таблицы в один датафрейм, чтобы выделить features ОДИНАКОВЫМ образом
и для тренировочного, и для тестового набора данных

In [ ]:
df = pd.concat([df_train,df_test])

Разворачиваем колонку параметров, так как в ней расположена еще куча features
поскольку это колонка словарей, ее легко превратить в новый dataFrame
также здесь все пропущенные значения (NaN) заменяются на -1, чтобы быть каким-то числом

In [ ]:
params = df['params'].apply(pd.Series)
params = params.fillna(-1)

if "Охрана" not in df:
    df = pd.concat([df,params],axis=1)

Попытка выбора признаков исходя из количества данных для них (попробовать признаки с более чем 400 вхождениями)
была не слишком удачная - больше информации было из блока оценки eli5 в стартере2. 
Тем не менее, интересно было посмотреть. В будущем лучше использовать heatmap по отсутствующим данным.

In [ ]:
feats_nunique = {feat:params[feat].nunique() for feat in params.columns}
z = {k:v for k,v in sorted(feats_nunique.items(),key =ig(1))}
caters = list(filter(lambda x: z[x] > 400,z.keys()))


итоговые категориальные признаки, которые не будут преобразованы, а сразу отправятся на факторизацию
они были определны частично по работе eli5 для простой модели дерева решений, частично подбором

In [ ]:
qualit = ["Новостройка:",'Тип балкона:','Класс жилья:']

блок географических признаков.
1. Из geo_block был выделен простой бинарный признак "Москва-не Москва"
2. Из breadcrumbs был выделен не очень хорошо сдавливаемый категориальный признак района.
Его бы в идеале превратить в ранговый (например, по удаленности от центра, в самом простом случае),
но автоматически сделать это возможности нет. 
Поэтому здесь данные лишь сглаживаются по форме (отрезается "метро", например) и отправляются на факторизацию
3. комбинированный ранговый признак - метро + МЦК. функции парсят колонку 'breadcrumbs' и возвращают 1, если есть
соответствующий элемент
для экономии кода одна и та же двухпараметрическая функция metro каррируется с параметром с помощью functools.partial

In [ ]:
def region(lst):
    if len(lst) >=2:
        if lst[1].startswith('м.'):
            return lst[1][3:]
        else:
            return lst[1]
    else:
        return -1

def metro(val,x):
    return int(any(x in k for k in val))
    
df['loc_cat']=df['breadcrumbs'].map(region)
df['loc_cat'] = df['loc_cat'].factorize()[0]

df['city_cat'] = df['geo_block'].map(lambda x: int(ig(0)(x) == 'г. Москва'))
df['metro_cat'] = df['breadcrumbs'].map(partial(metro,x='м. ')) + df['breadcrumbs'].map(partial(metro,x='МЦК'))

4. признак охраны был введен на ранговой основе. Функция security парсит значения из колонки "Охрана"
и при нахождении определенных кусков слов добавляет к итоговому результату какое-то количество баллов. 
В результате, чем охрана навороченнее - тем выше итоговое значение признака

In [ ]:
def security(val):
    if val == -1:
        return 0
    sm = 0
    rest = {'огорожен','огра','закры','кпп','доступ'}
    if any(w in val for w in rest):
        sm += 1
    cam = {'видео','камер'}
    if any(w in val for w in cam):
        sm += 1
    if 'консьерж' in val:
        sm += 1
    if 'круглосут' in val:
        sm += 1
    return sm 


df['secur_cat']=df['Охрана:'].map(security)

5. блок парковки изначально состоял из двух признаков, очевидно следующих из предлагаемых в колонке данных:
числа машиномест и типа парковки. 
Функции parking_spaces и parking_loc парсят данные из колонки "Парковка":; первая находит максимум из чисел, 
лежащих в строке данных - это и есть число машиномест (если оно там есть), и возвращает его натуральный логарифм,
чтобы разброс данных был меньше и модель кушала лучше
вторая функция была призвана ранжировать признак по наличию подземной или наземной парковки, или обоих типов
Потом по результатам тестирования второй признак оказался не полезным для результата, поэтому не использовался - 
остался только первый
6. в типе здания были просто порезаны окончания, чтобы число вариантов было меньше. 
можно было бы статистически проанализировать веса категорий по гистограмме, но распределение относительно цены само по 
себе все равно такое, что сильного влияния не оказывает. сам признак полезный, влияет на ответ

In [ ]:
def parking_spaces(val):
    if val!=-1 and any(l.isdigit() for l in val.split()):
        v = val.split()
        num = max(list(filter(str.isdigit,v)),key=int)
        return np.log(int(num)) if num else -1
    else:
        return -1
    
def parking_loc(val):
    if val!=-1:
        res = ''
        if 'подзем' in val:
            res += '1'
        if 'назем' or 'открыта' in val:
            res += '2'
        if len(res) == 0:
            return -1
        elif len(res) == 1:
            return int(res)
        else:
            return 3
    else:
        return val
    
def edifice(val):
    if val != -1:
        v = val.lower()[:-2]
    return -1
         
df['parkspaces_cat'] = df['Парковка:'].map(parking_spaces)
#df['parkloc_cat'] = df['Парковка:'].map(parking_loc)
df['type_cat'] = df['Тип здания:'].map(edifice).factorize()[0]

7-11. набор остальных признаков из блока params. Все из них пропарсены; признак "Этаж:" разделен на два - собственно этаж
квартиры и этажность здания. 
Все признаки были протестированы, и оставлены только те, включение которых улучшало результат обучения.
Полезными оказались: этаж,этажность,общая площадь, количество комнат, высота потолков

In [ ]:
def parse_end(val,en):
    return float(val[:en]) if val != -1 else int(val)

def floor(val):
    if type(val)==str and '/' in val:
        return list(map(int,val.split('/')))
    else:
        return int(val),-1

def rooms(val):
    if int(val) < 9:
        return int(val)
    else:
        return 9
    
#df['kitchen_cat'] = df['Площадь кухни:'].map(partial(parse_end,en=-3))
df['floor_cat'],df['height_cat'] = zip(*df['Этаж:'].map(floor))
df["area_cat"] = df["Общая площадь:"].map(lambda x: float(x[:-3]))
df['rooms_cat'] = df['Количество комнат:'].map(rooms)
df['ceiling_cat'] = df['Высота потолков:'].map(partial(parse_end,en=-2))
#df['living_cat'] = df['Жилая комната:'].map(partial(parse_end,en=-3))

Небольшой блок фильтрации данных от выбросов

In [ ]:
df = df[df['area_cat']<1000]
df = df[df['ceiling_cat'] != 100.0]

Зафейленная идея парсить дату публикации. сами данные настолько неоднородные, что выгоднее вообще оказалось
исключить признак из рассмотрения. но технически сделать было интересно

In [ ]:
MONTHS = {'января':'01','февраля':'02','марта':'03','апреля':'04','мая':'05','июня':'06','июля':'07',
         'августа':'08','сентября':'09','октября':'10','ноября':'11','декабря':'12'}
def datefilter(st):
    if st[0].isdigit():
        v = st.split()
        d,m = v[0],v[1]
        y = v[2] if len(v)>2 else '2018'
        return int(MONTHS[m]),y
    else:
        return -1,-1

#df['month_cat'],df['year_cat'] = zip(*df['Дата публикации:'].apply(datefilter))

Блок факторизации категориальных признаков, выбранных ранее

In [ ]:
for feat in qualit:
    df[f'{feat}_cat'] = df[feat].factorize()[0]

Окончательная выборка признаков для обучения - это все столбцы,имеющие _сat в названии
а далее - странный трюк. Чтобы модели было чуть легче отделять отсутствие данных, сделаем его 
более отстоящим от основных групп данных, заменив -1 на -100.

In [ ]:
cat_feats = list(filter(lambda x: '_cat' in x,df.columns))

df[cat_feats] = df[cat_feats].replace(-1,-100)

Разделим уже подготовленный датафрейм обратно на обучающую и тестовую выборки по наличию значения в столбце price

In [ ]:
df_train = df[~df['price'].isnull()].copy()
df_test = df[df['price'].isnull()].copy()
df_train.shape, df_test.shape

Cформируем наборы данных для обучения и теста по подготовленному ранее списку категорий cat_feats - всего у нас 14 признаков!
Слава прихотливой индексации в pandas! как же это удобно

In [ ]:
X_train = df_train[cat_feats]
y_train = df_train['price']

X_test = df_test[cat_feats]
X_train

Самое главное - собственно выбор модели и ее параметров.
Очевидно, наверное, что простая модель регрессии на такой разнородной, дырявой и обширной выборке будет не слишком эффективна.
Поэтому выбор стоит сделать в пользу ансамблей моделей - они лежат или в sklearn.ensemble, или реализованы
в сторонних библиотеках типа XGBoost
Вариантов ансамблевых моделей по-простому три: бэггинг, бустинг и стекинг.
Первый строит множество однородных моделей на поднаборах обучающих данных и потом обучается на их результатах
Второй "прокачивает работу" ансамбля слабых моделей, итеративно обучая каждый следующий шаг ансамбля на ошибках предыдущего шага
Третий объединяет работу разнородных моделей в "стопку" так, чтобы они компенсировали слабые места друг друга

Поскольку конечным желаемым вариантом работы от нас требуют mae - mean absolute error, т.е. смещение, вариантов выбора ансамблей
два - или бэггинг, или бустинг. Стекинг направлен на уменьшение разброса результатов работы модели (кстати, это работает
на данной выборке!), поэтому здесь не совсем подходит. Я выбрал подход бэггинга.

Поскольку мы уже работали с деревьями решений, логично в качестве основного эстиматора для ансамбля выбрать дерево -
то есть или DecisionTreeRegressor, или его более рандомизированный вариант ExtraTreeRegressor из sklearn.tree

Это оставляет перед следующими вариантами:
- конвенционный бэггинг в sklearn.ensemble.BaggingRegressor c заданием или DecisionTree, или ExtraTree в качестве базы
- случайный лес в sklearn.ensemble.RandomForestRegressor
- экстремальный случайный лес в sklearn.ensemble.ExtraTreesRegressor

Протестировав все, оказалось, что ExtraTreesRegressor дает наилучшие результаты за вполне разумное время (порядка 13-15 секунд)
с заданными параметрами. К слову о них - количество эстиматоров было выбрано оптимальным (больше, меньше = хуже, а в случае 
больше еще и дольше =)), max_features позволяет взять не все признаки разом для обучения, а только часть из них. Взяв порядка 
корня из 14 (у нас ведь 14 признаков) получаем чуть больше 3 - именно это число и оказалось оптимальным, увеличив быстродействие
работы модели без потери эффективности.

In [ ]:
model = ExtraTreesRegressor(n_estimators=77, random_state=0, max_features=3)
scores = cross_val_score(model,X_train,y_train,cv=5,scoring='neg_mean_absolute_error')
np.mean(scores),np.std(scores)

In [ ]:
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

In [ ]:
df_test['price'] = y_pred
print(df_test.shape)
df_test[['id','price']].to_csv("../output/os14.csv",index=False)

Средняя относительная ошибка - отношение MAE к средней цене по выборке. Параметр чисто для себя, как более привычный

In [ ]:
mean_relative_error = np.mean(scores)/np.mean(df_test['price']) * 100
mean_relative_error